In [1]:
import ipywidgets as widgets
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.interactive.artists import ScatterArtist
import numpy as np

In [2]:
from ipyevents import Event
from scipy.spatial import KDTree

In [59]:
class EditPointTags:

    def __init__(self, tags, data_fields=None):
        self._tags = tags

        if data_fields is None:
            data_fields = {}

        self._data_fields = data_fields
        self._event = Event(watched_events=['click'])

    @property
    def label(self):
        return self._label

    @label.setter
    def label(self, value):
        self._label = value

    def activate(self, canvas):
        self._event.source = canvas

        def handle_event(event):
            x, y = canvas.pixel_to_domain(event['offsetX'], event['offsetY'])
            tags = self._tags

            if event['altKey'] is True:
                positions = np.array([tags.x, tags.y]).T
                distances, indices = KDTree(positions).query([[x, y]])
                tags.delete_tags(indices)
            else:
                tags.add_tags([x], [y], {key: [value] for key, value in self._data_fields.items()})

        self._event.on_dom_event(handle_event)

    def deactivate(self, canvas):
        self._event.reset_callbacks()


In [8]:
canvas2.x_scale

LinearScale(allow_padding=False, max=10.0, min=0.0)

In [142]:
canvas1 = Canvas()
canvas2 = Canvas()

fft_artist = ScatterArtist()
canvas2.artists = {'artist': fft_artist}

canvas2.x_limits = [0,10]
canvas2.y_limits = [0,10]

fft_artist.x = np.linspace(0,10,10)
fft_artist.y = np.linspace(0,10,10)
#fft_artist._mark.enable_move = True
#fft_artist._mark.restrict_y = True
fft_artist._mark.default_size = 200


event = Event(watched_events=['click', 'mousemove'])


l = widgets.Label('Click or type on me!')
l.layout.border = '2px solid red'

h = widgets.HTML('Event info')
event_obj = Event(source=canvas2, watched_events=['click', 'mousemove', 'mouseup', 'mousedown'])

event_obj._selected = None

def handle_event(event):
    mouse_x, mouse_y = canvas2.pixel_to_domain(event['offsetX'], event['offsetY'])
    
    if event['type'] == 'mousedown':
        positions = np.array([fft_artist.x, fft_artist.y]).T
        distances, indices = KDTree(positions).query([[mouse_x, mouse_y]])
        event_obj._selected = indices[0]
    elif event['type'] == 'mousemove':
        if event_obj._selected is not None: 
            x = fft_artist.x.copy()
            y = fft_artist.y.copy()
            color = fft_artist._mark.color.copy()
            
            y[event_obj._selected] = mouse_y
            color[event_obj._selected] = -(x[event_obj._selected] - mouse_x) * 5

            h.value = f'{event_obj._selected}'

            fft_artist.x = x
            fft_artist.y = y
            fft_artist._mark.color = color
            
    elif event['type'] == 'mouseup':
        event_obj._selected = None
    
fft_artist._mark.color = np.linspace(0,10,10)

event_obj.on_dom_event(handle_event)

widgets.VBox([widgets.HBox([canvas1, canvas2]), h])

In [124]:
event_obj._selected

4

Label(value='Click or type on me!', layout=Layout(border='2px solid red'))

HTML(value='Event info')